# HCMUS-UNIV-WebScience-VisualizeData

## PROJECT 2: MỐI QUAN HỆ CỦA DỮ LIỆU


### I. Chuẩn bị môi trường:

In [232]:
import sqlite3
import pandas as pd
import json
import os

In [233]:
def fetch_value(sqlCommand):
    conn = sqlite3.connect("data.db",check_same_thread=False)
    ret = pd.read_sql_query(sqlCommand, conn)
    conn.close()
    return ret

# Query dữ liệu
def query_sql(sql:str):
    listData = fetch_value(sql)
    return listData

# Lấy bài viết theo website.
def get_bai_viet(website:str):
    return query_sql(f"select * from baiviet where website='{website}'")

class Utils:
    def convertHMtoFloat(HM):
        dv = str(HM).split(":")
        if len(dv) > 1:
            h,m = dv
        else:
            h,m = dv[0],0
        return int(h)+(float(m)/60)

### I. Tiền xử lý dữ liệu

 #### Yêu cầu: đưa các field về dạng chuẩn chung:

1. active_time: thời gian hoạt động, đưa về dạng sau:

```
    {
        <tên ngày trong tuần>:[ <thời gian mở cửa>,<thời gian đóng cửa> ]
    }

    *lowercase
    *thời gian đưa về float: 8h30 => 8.5
```
    
- ví dụ:

```
    {
        "thứ 2": [7,21.5],
        ...,
        "chủ nhật": [8,22.3]
    }
```

2. other_service: các dịch vụ khác, đưa về dạng sau:

```
    [
        <tên dịch vụ>,
        <tên dịch vụ>,
        ...,
    ]
    
    *lowercase
```
    
- ví dụ:

```
    [
        "có wifi"
        "giao hàng",
        ...,
    ]
```

#### I.1 Foody

In [234]:
def preprocessing_Foody():
    # Code here
    data = get_bai_viet("Foody")
    active_time = data["active_time"]
    other_service = data["other_service"]
    format_time = []
    format_other = []
    for time in active_time:
        #time = time.open + ',' + time.close
        time = json.loads(time)
        time_format = []
        time_format.append(Utils.convertHMtoFloat(time['open']))
        time_format.append(Utils.convertHMtoFloat(time['close']))
        x = {
            "thứ 2":time_format,
            "thứ 3":time_format,
            "thứ 4":time_format,
            "thứ 5":time_format,
            "thứ 6":time_format,
            "thứ 7":time_format,
            "chủ nhật":time_format,
        }
        format_time.append(x)
    data['active_time'] = format_time
    for other in other_service:
        other = other.lower()
        other = json.loads(other)
        format_other.append(other)
    data['other_service'] = format_other

    def reformat_district_map(district):
        return district.replace("huyện ","").replace("thành phố ","")
    data['district'] = data['district'].apply(reformat_district_map)

    return data

#### I.2 ShopeeFood

In [235]:
def preprocessing_ShopeeFood():
    def maping(name):
        if name=="monday":
            return "thứ 2"
        if name=="tuesday":
            return "thứ 3"
        if name=="wednesday":
            return "thứ 4"
        if name=="thursday":
            return "thứ 5"
        if name=="friday":
            return "thứ 6"
        if name=="saturday":
            return "thứ 7"
        if name=="sunday":
            return "chủ nhật"

    def convert_active_time(HM):
        pre_format = {}
        at_js = json.loads(HM)
        atl = list(at_js)
        for key in atl:
            open,close = at_js[key].split("-")
            pre_format[maping(key)] = [Utils.convertHMtoFloat(open),Utils.convertHMtoFloat(close)]
        return pre_format

    data = get_bai_viet("ShopeeFood")
    data["active_time"] = data["active_time"].apply(convert_active_time)

    return data

#### I.3 GoogleReview

In [236]:
def preprocessing_GoogleReview():
    def reformat_district_map(district):
        return district.replace("thành phố ","")
    
    BlackDistrict=[
        '2 hải triều',
        'vn  1 45 lý tự trọng bến nghé',
        'district 1',
        'thảo điền  2 hồ chí minh',
        'district 02',
        'phường 10  4 hồ chí minh',
        '06',
        '05',
        '19a đường 182 lã xuân oai tăng nhơn phú a',
        'số 01 đường 48 kdc bảo châu phường 16  08 tp',
        'tp',
        '30 cửu long phường 15  10  10',
        '479/55 tân thới hiệp 07',
    ]

    def maping(name):
        if name=="Thứ Hai":
            return "thứ 2"
        if name=="Thứ Ba":
            return "thứ 3"
        if name=="Thứ Tư":
            return "thứ 4"
        if name=="Thứ Năm":
            return "thứ 5"
        if name=="Thứ Sáu":
            return "thứ 6"
        if name=="Thứ Bảy":
            return "thứ 7"
        if name=="Chủ Nhật":
            return "chủ nhật"

    def convert_active_time(HM):
        pre_format = {}
        at_js = json.loads(HM)
        atl = list(at_js)
        for key in atl:
            if (key == "open"):
                pre_format = {
                    "thứ 2":[Utils.convertHMtoFloat("00:00"),Utils.convertHMtoFloat("23:59")],
                    "thứ 3":[Utils.convertHMtoFloat("00:00"),Utils.convertHMtoFloat("23:59")],
                    "thứ 4":[Utils.convertHMtoFloat("00:00"),Utils.convertHMtoFloat("23:59")],
                    "thứ 5":[Utils.convertHMtoFloat("00:00"),Utils.convertHMtoFloat("23:59")],
                    "thứ 6":[Utils.convertHMtoFloat("00:00"),Utils.convertHMtoFloat("23:59")],
                    "thứ 7":[Utils.convertHMtoFloat("00:00"),Utils.convertHMtoFloat("23:59")],
                    "chủ nhật":[Utils.convertHMtoFloat("00:00"),Utils.convertHMtoFloat("23:59")],
                    }
            if (at_js[key] == "Mở cửa cả ngày" or at_js[key] == "Đóng cửa"):
                open = "00:00"
                close = "23:59"
            else :
                time_format = at_js[key].split("–")
                open = time_format[0]
                close = time_format[len(time_format) - 1]
            
            pre_format[maping(key)] = [Utils.convertHMtoFloat(open),Utils.convertHMtoFloat(close)]
        return pre_format

    data = get_bai_viet("GoogleReview")
    data["active_time"] = data["active_time"].apply(convert_active_time)
    
    def convert_other_service(strjson):
        strjson = strjson.replace('\\u00a0','').lower()
        return json.loads(strjson)
    data['other_service'] = data['other_service'].apply(convert_other_service)

    data['district'] = data['district'].apply(reformat_district_map)

    data = data[ ~(data['district'].isin(BlackDistrict)) ]

    return data

#### I.4 Tổng hợp lại dữ liệu tiền xử lý

- Thực hiện kết hợp
- Phân rã dữ liệu thành dạng phẳng

In [237]:
def reformat(filename):
    DATA = pd.concat([preprocessing_Foody(),preprocessing_ShopeeFood(),preprocessing_GoogleReview()],ignore_index=True)

    # Chuyển complex type sang dạng giản đơn
    # active_time => thứ 2 on/off,...
    maping = {
        "thứ 2":"monday",
        "thứ 3":"tuesday",
        "thứ 4":"wednesday",
        "thứ 5":"thursday",
        "thứ 6":"friday",
        "thứ 7":"saturday",
        "chủ nhật":"sunday",
    }

    for index, row in DATA['active_time'].items():
        for weekday in maping.keys():
            off,on = None,None
            if row.get(weekday):
                off,on = row[weekday]
            DATA.loc[index,maping[weekday]+'_off'] = off
            DATA.loc[index,maping[weekday]+'_on'] = on

    # liệt kê ra toàn bộ other_service
    other_services = set()
    for _, row in DATA['other_service'].items():
        if row:
            other_services = set.union(other_services, row)
    other_services = sorted(other_services)

    for index, row in DATA['other_service'].items():
        for kw in other_services:
            value = False
            if row and kw in row:
                value = True
            DATA.loc[index,'s_'+kw] = value
    DATA.to_csv(filename,index=False)
    return DATA

In [238]:
if os.path.exists('data_reformat.csv'):
    DATA = pd.read_csv('data_reformat.csv')
else:
    DATA = reformat('data_reformat.csv')

# Lấy ra các dịch vụ
SERVICES = []

cols = list(DATA.columns)
for col in cols:
    if col.startswith('s_'):
        SERVICES.append(col[2:])

### 2. Thống kê dữ liệu

BIẾN: 

- DATA: `dataframe` chứa toàn bộ dữ liệu
- SERVICES: `list` các dịch vụ

DANH SÁCH CÁC DỊCH VỤ:

In [239]:
for id, v in enumerate(SERVICES):
    if id % 5 == 0:
        print()
    print(repr(v),end='\t')
print("\n\n")
print("Có tổng cộng: %d dịch vụ" % len(SERVICES))


'cho mua về'	'có bàn ngoài trời'	'có chiếu bóng đá'	'có chỗ chơi cho trẻ em'	'có chỗ đậu ôtô'	
'có giao hàng'	'có giao hàng\n:\n> 100k & quận 1'	'có hồ bơi'	'có hỗ trợ hội thảo'	'có hỗ trợ người khuyết tật'	
'có karaoke'	'có khu vực hút thuốc'	'có lò sưởi'	'có máy lạnh & điều hòa'	'có nhạc sống'	
'có phòng riêng'	'có thẻ thành viên'	'có thẻ thành viên\n:\ncó bán vouchers'	'có wifi'	'có xuất hóa đơn đỏ'	
'có xuất hóa đơn đỏ\n:\nthẻ giảm giá'	'dịch vụ tại chỗ'	'giao hàng'	'giao hàng gián tiếp'	'giữ xe máy miễn phí'	
'giữ xe máy miễn phí\n:\n2 tiếng'	'không có đồ ăn mang đi'	'không giao hàng'	'không ăn tại chỗ'	'mua hàng ngay trên xe'	
'mua sắm tại cửa hàng'	'nhà hàng'	'nhận hàng ở lề đường'	'nhận tại cửa hàng'	'nên đặt trước'	
'nên đặt trước\n:\nbắt buộc'	'phòng gym'	'spa & massage'	'sân tennis'	'tip cho nhân viên'	
'trả bằng thẻ'	'trả bằng thẻ\n:\nvisa/master'	'Ăn tại chỗ'	'Đặt lịch hẹn trực tuyến'	'Đồ Ăn tận phòng'	
'Đồ ăn mang đi'	


Có tổng cộng: 46 dịch vụ


#### Thống kê trên toàn bộ dữ liệu
- Các field: rate, rate_count, price_from, price_to

In [240]:
#Summary statistic

DATA.describe()

,rate,rate_count,favorite,price_from,price_to,monday_off,monday_on,tuesday_off,tuesday_on,wednesday_off,wednesday_on,thursday_off,thursday_on,friday_off,friday_on,saturday_off,saturday_on,sunday_off,sunday_on
count,5801.000000,5801.000000,3927.000000,5.801000e+03,5.801000e+03,5783.000000,5783.000000,5787.000000,5787.000000,5787.000000,5787.000000,5785.000000,5785.000000,5786.000000,5786.000000,5783.000000,5783.000000,5761.000000,5761.000000
mean,5.819203,62.664388,41.939649,5.825490e+04,4.733293e+05,8.453139,20.179587,8.464677,20.185260,8.474662,20.175917,8.474898,20.177148,8.468839,20.172834,8.438723,20.182227,8.405164,20.201892
std,3.676943,153.641717,163.079957,1.455547e+05,2.248216e+06,4.418259,5.221462,4.414098,5.207387,4.411955,5.215470,4.410622,5.215819,4.412386,5.225363,4.436966,5.233537,4.475502,5.247239
min,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,6.500000,20.000000,7.000000,20.000000,7.000000,20.000000,7.000000,20.000000,7.000000,20.000000,6.500000,20.000000,6.500000,20.000000
50%,7.586000,3.000000,2.000000,2.000000e+04,1.000000e+05,9.000000,22.000000,9.000000,22.000000,9.000000,22.000000,9.000000,22.000000,9.000000,22.000000,9.000000,22.000000,9.000000,22.000000
75%,8.600000,32.000000,17.000000,5.500000e+04,3.000000e+05,10.000000,23.000000,10.000000,23.000000,10.000000,23.000000,10.000000,23.000000,10.000000,23.000000,10.000000,23.000000,10.000000,23.000000
max,10.000000,1000.000000,4746.000000,4.000000e+06,1.000000e+08,22.000000,23.983333,22.000000,23.983333,22.000000,23.983333,22.000000,23.983333,22.000000,23.983333,22.000000,23.983333,22.000000,23.983333


#### Thống kê theo điều kiện 1

In [241]:
# Số lượng từng website
DATA.groupby(['website'])['website'].count()

website
Foody           3027
GoogleReview    1874
ShopeeFood       900
Name: website, dtype: int64

- Tiêu chí rate, rate_count, price_from, price_to

In [242]:
colList = ['rate', 'rate_count', 'price_from', 'price_to']

In [243]:
# Thống kê theo từng website
print(DATA[DATA['website']=='Foody'][colList].describe())
print()
print(DATA[DATA['website']=='GoogleReview'][colList].describe())
print()
print(DATA[DATA['website']=='ShopeeFood'][colList].describe())

              rate   rate_count    price_from      price_to
count  3027.000000  3027.000000  3.027000e+03  3.027000e+03
mean      4.785596    12.270895  6.715101e+04  3.017703e+05
std       3.739141    36.984187  1.785913e+05  9.131174e+05
min       0.000000     0.000000  0.000000e+00  0.000000e+00
25%       0.000000     0.000000  1.000000e+04  3.000000e+04
50%       6.680000     1.000000  2.500000e+04  1.200000e+05
75%       7.917000     7.000000  5.000000e+04  2.990000e+05
max      10.000000   584.000000  4.000000e+06  2.700000e+07

              rate   rate_count     price_from        price_to
count  1874.000000  1874.000000    1874.000000     1874.000000
mean      7.832764   145.714043   40661.686233   107310.565635
std       1.563947   222.345001   91033.044666   203085.969412
min       2.000000     0.000000       0.000000        0.000000
25%       7.600000     2.000000       0.000000        0.000000
50%       8.200000    26.000000       0.000000        0.000000
75%       8.800000

In [244]:
# Thống kê theo từng quận
districtList = sorted(DATA['district'].drop_duplicates().tolist())
for distr in districtList:
    print("Thống kê theo quận %s"%distr)
    print()
    print(DATA[DATA['district']==distr][colList].describe())
    print()

Thống kê theo quận 1

             rate   rate_count    price_from      price_to
count  904.000000   904.000000  9.040000e+02  9.040000e+02
mean     6.048867    73.729532  1.040498e+05  7.267737e+05
std      3.466038   170.237410  1.907290e+05  2.020570e+06
min      0.000000     0.000000  0.000000e+00  0.000000e+00
25%      5.000000     0.000000  2.000000e+04  8.900000e+04
50%      7.600000     5.000000  5.000000e+04  3.000000e+05
75%      8.400000    43.250000  1.000000e+05  5.122500e+05
max     10.000000  1000.000000  4.000000e+06  2.800000e+07

Thống kê theo quận 10

             rate   rate_count     price_from      price_to
count  393.000000   393.000000     393.000000  3.930000e+02
mean     6.355354   105.631837   45243.002545  5.112830e+05
std      3.471307   205.718532   72875.663669  3.289135e+06
min      0.000000     0.000000       0.000000  0.000000e+00
25%      5.000000     0.000000       0.000000  0.000000e+00
50%      7.800000    10.000000   20000.000000  5.500000e+04
75%

In [245]:
# Thống kê theo rate
tmp = ['rate_count', 'price_from', 'price_to']

print("Thống kê những nhà hàng tốt rate > 8 sao")
print(DATA[DATA['rate']>8][tmp].describe())
print()
print("Thống kê những nhà hàng tệ rate < 3 sao")
print(DATA[DATA['rate']<3][tmp].describe())

Thống kê những nhà hàng tốt rate > 8 sao
        rate_count    price_from      price_to
count  2209.000000  2.209000e+03  2.209000e+03
mean    109.500197  5.858967e+04  3.409351e+05
std     198.811602  1.634651e+05  7.865392e+05
min       1.000000  0.000000e+00  0.000000e+00
25%       3.000000  0.000000e+00  0.000000e+00
50%      12.000000  2.000000e+04  1.000000e+05
75%     100.000000  7.000000e+04  3.000000e+05
max    1000.000000  4.000000e+06  1.000000e+07

Thống kê những nhà hàng tệ rate < 3 sao
        rate_count    price_from      price_to
count  1530.000000  1.530000e+03  1.530000e+03
mean      0.083007  5.395690e+04  1.001300e+06
std       0.648214  1.435519e+05  4.193030e+06
min       0.000000  0.000000e+00  0.000000e+00
25%       0.000000  0.000000e+00  0.000000e+00
50%       0.000000  2.000000e+04  6.000000e+04
75%       0.000000  5.000000e+04  4.575000e+05
max      13.000000  2.000000e+06  1.000000e+08


#### Thống kê theo điều kiện 2

In [ ]:
# Code here
DATA.sort_values(by="rate", ascending=False).head()
#Những quán ăn Quận 1 mà được lượt rate cao tới thấp
DATA[DATA["district"].apply(lambda state: state[0] == "1")].sort_values(by="rate", ascending=False).head()

#Những quán có rate cao là số favorite cũng cao
columns_to_show = ["rate", "favorite", "district"]
DATA.sort_values(by="rate", ascending=False).groupby(["full_name"])[columns_to_show].describe(percentiles=[])

#Những quán có giá đắt
columns_show = ["full_name","price_from","price_to",]
DATA.sort_values(by="price_from",ascending=False).groupby(["full_name"])[columns_show].describe(percentiles=[])

###  Phần 3,4,5